In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [32]:
import cv2
import pickle
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from keras.models import model_from_json
from keras.applications.xception import preprocess_input as pi_xception
from keras.applications.inception_resnet_v2 import preprocess_input as pi_irnv2

In [33]:
PATH_W = '/content/gdrive/My Drive/ensemble_normalized_weights.pkl'
PATH_THRESH = '/content/gdrive/My Drive/thresholds.npy'
with open(PATH_W, 'rb') as fp:
    NORM_WEIGHTS = pickle.load(fp)
THRESHOLDS = np.load(PATH_THRESH)

In [34]:
MODEL_NAMES = ['xception', 'irnv2', 'dn121', 'dn169', 'dn201']

PATHS_ARCH = {
    'xception' : '/content/gdrive/My Drive/Model-Architectures/architecture_xception.json',
    'irnv2'    : '/content/gdrive/My Drive/Model-Architectures/architecture_irnv2.json',
    'dn121'    : '/content/gdrive/My Drive/Model-Architectures/architecture_dn121.json',
    'dn169'    : '/content/gdrive/My Drive/Model-Architectures/architecture_dn169.json',
    'dn201'    : '/content/gdrive/My Drive/Model-Architectures/architecture_dn201.json'
}

PATHS_WEIGHTS = {
    'xception' : '/content/gdrive/My Drive/Model-Weights/xception.h5',
    'irnv2'    : '/content/gdrive/My Drive/Model-Weights/irnv2.h5',
    'dn121'    : '/content/gdrive/My Drive/Model-Weights/dn121.h5',
    'dn169'    : '/content/gdrive/My Drive/Model-Weights/dn169.h5',
    'dn201'    : '/content/gdrive/My Drive/Model-Weights/dn201.h5'
}

PREPROCESS_INPUTS = {
    'xception' : pi_xception,
    'irnv2'    : pi_irnv2,
    'dn121'    : None,
    'dn169'    : None,
    'dn201'    : None
}

MODELS = {
    'xception' : None,
    'irnv2'    : None,
    'dn121'    : None,
    'dn169'    : None,
    'dn201'    : None
}

In [35]:
 def build_models():
  for model_name in MODEL_NAMES: 
      json_file = open(PATHS_ARCH[model_name], 'r')
      model_architecture = json_file.read()
      json_file.close()
      model = model_from_json(model_architecture)
      model.load_weights(PATHS_WEIGHTS[model_name])
      MODELS[model_name] = model

In [36]:
def preprocess(raw_img, preproc_func = None):
  img = cv2.resize(raw_img, (224, 224))
  if preproc_func != None:
    img = preproc_func(img)
  else: 
    img = img/255.
  img = img.reshape(1, 224, 224, 3)
  return img

In [37]:
def get_probabilities(raw_img):
  predictions = {
      'xception' : None,
      'irnv2'    : None,
      'dn121'    : None,
      'dn169'    : None,
      'dn201'    : None
  }
  for model_name in MODEL_NAMES: 
    img = preprocess(raw_img, preproc_func = PREPROCESS_INPUTS[model_name])
    model = MODELS[model_name]
    predictions[model_name] = model.predict(img)[0]
  return predictions

In [38]:
build_models()

In [56]:
IMG_PATH = '/content/img.png'
raw_img = cv2.imread(IMG_PATH)

probabilities = get_probabilities(raw_img)

for model_name in ['dn169', 'dn201']:
  temp = np.empty(probabilities[model_name].shape)
  temp[0] = probabilities[model_name][0]
  temp[1] = probabilities[model_name][2]
  temp[2] = probabilities[model_name][3]
  temp[3] = probabilities[model_name][1]
  temp[4] = probabilities[model_name][4]
  probabilities[model_name] = temp

sum = 0
for model_name in MODEL_NAMES:
  sum += probabilities[model_name] * NORM_WEIGHTS[model_name]
ensembled_predictions = sum/len(MODEL_NAMES)
y_hat = ensembled_predictions > THRESHOLDS
print(y_hat)

[False False False False False]
